# Analysis of AMR genes

#### Author: Liubov Chuprikova

In [1]:
# import modules
import os
import pandas as pd
import numpy as np
from prettytable import PrettyTable
import sys
sys.getdefaultencoding()

'utf-8'

In [2]:
SUMMARIES_DIR = '/home/liubov/Documents/tfm/the_whole_analysis/abricate_analysis/summaries'

def make_megares_tidy(megares_df):
    tidy_df = pd.DataFrame(columns=['STRAIN', 'ID', 'TYPE', 'CLASS', 'MECHANISM', 'GROUP'])
    for _, row in megares_df.iterrows():
        entry = row['#FILE']
        index_array = np.flatnonzero(row.iloc[2:].notna()) + 2
        entry_types = [row.index[index] for index in index_array]
        for etype in entry_types:
            levels = etype.split(sep='|')
            data = pd.DataFrame({'STRAIN': [entry], 'ID': [levels[0]], 'TYPE': [levels[1]],
                            'CLASS': [levels[2]], 'MECHANISM': [levels[3]], 'GROUP': [levels[4]]})
            tidy_df = tidy_df.append(data, ignore_index=True)

    return tidy_df

def make_tidy(df):
    tidy_df = pd.DataFrame(columns=['STRAIN', 'GENE'])
    for _, row in df.iterrows():
        entry = row['#FILE']
        index_array = np.flatnonzero(row.iloc[2:].notna()) + 2
        entry_types = [row.index[index] for index in index_array]
        for etype in entry_types:
            data = pd.DataFrame({'STRAIN': [entry], 'GENE': [etype]})
            tidy_df = tidy_df.append(data, ignore_index=True)
    
    return tidy_df

def get_sheared_entities(entities, entity_name, grouped_data):
    sheared_entities = []
    for entity in entities:
        count = 0
        for _, group in grouped_data:
            if entity in group[entity_name].unique().tolist():
                count += 1
        if count > len(grouped_data) * 0.95:
            sheared_entities.append(entity)

    return sheared_entities

### MEGARes

`megares_noSNPs` is a database that was downloaded from MEGARes repository (https://megares.meglab.org/download/index.php) -- Version 2.0.0 (14 October 2019). Genes that requires SNP confirmation were excluded (`ExplorePipolin/scripts/prepare_megares_nosnps.py`).

**MEGARes is non-redundant and includes entities from 5 databases: BacMet, ResFinder, ARG-ANNOT, CARD and NCBI AMR.**

Each entry (gene) in this database is represented by it's unique ID and 4 hierarchical levels: TYPE, CLASS, MECHANISM and GROUP (by analogy with GO ontology).

 * **Type** of compound, to which the accession confers resistance (e.g. drug, biocide, metal, multi-compound),
 * **Class** of antimicrobial compounds to which a gene confers resistance (e.g. betalactams),
 * **Mechanism** by which this resistance is conferred (e.g. betalactamases),
 * **Group** name of the genes (e.g. Group A betalactamases),
 * and **ID** for each individual gene accession.

In [3]:
megares_summary = pd.read_csv(os.path.join(SUMMARIES_DIR, 'megares_noSNPs.tab'),
                              sep='\t', na_values='.')

In [4]:
megares_tidy = make_megares_tidy(megares_summary)
print(megares_tidy)

                STRAIN        ID            TYPE  \
0       AP019703.1.tab  MEG_1107        Biocides   
1       AP019703.1.tab  MEG_1164          Metals   
2       AP019703.1.tab  MEG_1180           Drugs   
3       AP019703.1.tab  MEG_1181        Biocides   
4       AP019703.1.tab  MEG_1189           Drugs   
...                ...       ...             ...   
15146  chr_LREC262.tab  MEG_7859          Metals   
15147  chr_LREC262.tab  MEG_7862          Metals   
15148  chr_LREC262.tab  MEG_7864          Metals   
15149  chr_LREC262.tab  MEG_7865  Multi-compound   
15150  chr_LREC262.tab  MEG_7867          Metals   

                              CLASS                               MECHANISM  \
0          Multi-biocide_resistance      Multi-biocide_resistance_regulator   
1            Multi-metal_resistance          Multi-metal_resistance_protein   
2             Multi-drug_resistance         Multi-drug_RND_efflux_regulator   
3                   Acid_resistance                 Acid_re

In [5]:
# total number of unique entities for each hierarchical level
total_ids = megares_tidy['ID'].unique().tolist()
total_types = megares_tidy['TYPE'].unique().tolist()
total_classes = megares_tidy['CLASS'].unique().tolist()
total_mechanisms = megares_tidy['MECHANISM'].unique().tolist()
total_groups = megares_tidy['GROUP'].unique().tolist()

x = PrettyTable()
x.field_names = ['--', 'ID', 'TYPE', 'CLASS', 'MECHANISM', 'GROUP']
x.add_row(['TOTAL IN ALL', len(total_ids), len(total_types), 
           len(total_classes), len(total_mechanisms), len(total_groups)])
print(x)

+--------------+-----+------+-------+-----------+-------+
|      --      |  ID | TYPE | CLASS | MECHANISM | GROUP |
+--------------+-----+------+-------+-----------+-------+
| TOTAL IN ALL | 289 |  4   |   34  |     76    |  234  |
+--------------+-----+------+-------+-----------+-------+


In [6]:
megares_tidy_grouped = megares_tidy.groupby(['STRAIN'])
print(len(megares_tidy_grouped) * 0.95)
# get entities that present in more than 95% of strains (89-93)
sheared_ids = get_sheared_entities(total_ids, 'ID', megares_tidy_grouped)
sheared_types = get_sheared_entities(total_types, 'TYPE', megares_tidy_grouped)
sheared_classes = get_sheared_entities(total_classes, 'CLASS', megares_tidy_grouped)
sheared_mechanisms = get_sheared_entities(total_mechanisms, 'MECHANISM', megares_tidy_grouped)
sheared_groups = get_sheared_entities(total_groups, 'GROUP', megares_tidy_grouped)


x.add_row(['SHEARED B/W STRAINS', len(sheared_ids), len(sheared_types), 
           len(sheared_classes), len(sheared_mechanisms), len(sheared_groups)])
print(x)

88.35
+---------------------+-----+------+-------+-----------+-------+
|          --         |  ID | TYPE | CLASS | MECHANISM | GROUP |
+---------------------+-----+------+-------+-----------+-------+
|     TOTAL IN ALL    | 289 |  4   |   34  |     76    |  234  |
| SHEARED B/W STRAINS | 136 |  4   |   22  |     45    |  140  |
+---------------------+-----+------+-------+-----------+-------+


In [7]:
print(sheared_ids)

['MEG_1164', 'MEG_1180', 'MEG_1181', 'MEG_1192', 'MEG_1193', 'MEG_1210', 'MEG_1231', 'MEG_1701', 'MEG_2017', 'MEG_2020', 'MEG_2087', 'MEG_2088', 'MEG_2091', 'MEG_2092', 'MEG_2132', 'MEG_2440', 'MEG_2443', 'MEG_2446', 'MEG_2447', 'MEG_2451', 'MEG_2454', 'MEG_2456', 'MEG_2457', 'MEG_2458', 'MEG_2688', 'MEG_2689', 'MEG_2692', 'MEG_2722', 'MEG_2725', 'MEG_2729', 'MEG_2732', 'MEG_2734', 'MEG_2736', 'MEG_2739', 'MEG_2867', 'MEG_2875', 'MEG_2898', 'MEG_2899', 'MEG_2909', 'MEG_3076', 'MEG_3078', 'MEG_3079', 'MEG_3080', 'MEG_3081', 'MEG_3083', 'MEG_3141', 'MEG_3253', 'MEG_3255', 'MEG_3271', 'MEG_3287', 'MEG_3289', 'MEG_3291', 'MEG_3448', 'MEG_3516', 'MEG_3622', 'MEG_3662', 'MEG_3663', 'MEG_3737', 'MEG_3746', 'MEG_3747', 'MEG_3749', 'MEG_3751', 'MEG_3752', 'MEG_3755', 'MEG_3756', 'MEG_3757', 'MEG_3759', 'MEG_3761', 'MEG_3763', 'MEG_3764', 'MEG_3765', 'MEG_3948', 'MEG_3998', 'MEG_3999', 'MEG_399', 'MEG_4000', 'MEG_4002', 'MEG_4006', 'MEG_4010', 'MEG_4012', 'MEG_401', 'MEG_4033', 'MEG_4061', 'MEG_

In [8]:
print(sheared_types)

['Biocides', 'Metals', 'Drugs', 'Multi-compound']


In [9]:
[i for i in sheared_classes]

['Multi-biocide_resistance',
 'Multi-metal_resistance',
 'Multi-drug_resistance',
 'Acid_resistance',
 'Bacitracin',
 'Drug_and_biocide_and_metal_resistance',
 'Drug_and_biocide_resistance',
 'Copper_resistance',
 'betalactams',
 'Sodium_resistance',
 'Cationic_antimicrobial_peptides',
 'Phenolic_compound_resistance',
 'Biocide_and_metal_resistance',
 'Peroxide_resistance',
 'Acetate_resistance',
 'Aminoglycosides',
 'MLS',
 'Chromium_resistance',
 'Nickel_resistance',
 'Arsenic_resistance',
 'Paraquat_resistance',
 'Zinc_resistance']

In [10]:
[i for i in sheared_mechanisms]

['Multi-biocide_resistance_regulator',
 'Multi-metal_resistance_protein',
 'Multi-drug_RND_efflux_regulator',
 'Acid_resistance_protein',
 'Undecaprenyl_pyrophosphate_phosphatase',
 'Drug_and_biocide_and_metal_RND_efflux_regulator',
 'Drug_and_biocide_MFS_efflux_pumps',
 'Copper_resistance_regulator',
 'Class_C_betalactamases',
 'Sodium_resistance_protein',
 'Copper_resistance_protein',
 'Drug_and_biocide_RND_efflux_regulator',
 'Multi-metal_RND_efflux_pumps',
 'Drug_and_biocide_MFS_efflux_regulator',
 'Lipid_A_modification',
 'Acid_resistance_regulator',
 'Phenolic_resistance_protein',
 'Multi-metal_ABC_efflux_pumps',
 'Biocide_and_metal_ABC_efflux_pumps',
 'Biocide_and_metal_resistance_protein',
 'Multi-drug_RND_efflux_pumps',
 'Peroxide_resistance_stress_protein',
 'Acetate_resistance_protein',
 'Aminoglycoside_efflux_pumps',
 'Drug_and_biocide_SMR_efflux_pumps',
 'Drug_and_biocide_and_metal_RND_efflux_pumps',
 'Drug_and_biocide_RND_efflux_pumps',
 'Drug_and_biocide_MATE_efflux_pump

In [11]:
print(sheared_groups)

['ARSCM', 'ASMA', 'ASR', 'BACA', 'BAER', 'BAES', 'BCR', 'BHSA', 'CHAA', 'COMR', 'COPA', 'CORA', 'CORB', 'CORC', 'CORD', 'CPXAR', 'CRP', 'CUEO', 'CUER', 'CUSA', 'CUSB', 'CUSF', 'CUTA', 'CUTC', 'CUTE', 'CUTF', 'DSBA', 'DSBB', 'DSBC', 'EMRA', 'EMRB', 'EMRD', 'EMRK', 'EMRR', 'EMRY', 'EPTA', 'EVGA', 'EVGS', 'FABI', 'FETA', 'FETB', 'FIEF', 'GADA', 'GADB', 'GADC', 'GADE', 'GADW', 'GADX', 'GLPF', 'HDEA', 'HDEB', 'HNS', 'IBPA', 'IBPB', 'ICLR', 'KDPE', 'KPNO', 'LPDT', 'MARA', 'MARR', 'MDFA', 'MDTA', 'MDTB', 'MDTC', 'MDTE', 'MDTF', 'MDTG', 'MDTH', 'MDTI', 'MDTJ', 'MDTK', 'MDTM', 'MDTN', 'MDTO', 'MDTP', 'MGTA', 'MNTH', 'MNTP', 'ACRA', 'MNTR', 'MODA', 'MODB', 'MODC', 'ACRB', 'MPHB', 'MSBA', 'ACRD', 'NFSA', 'NHAA', 'NHAB', 'NIKA', 'NIKB', 'NIKC', 'NIKD', 'NIKE', 'NIKR', 'OXYRKP', 'PBP2', 'PBP4B', 'PITA', 'PMRF', 'PSTA', 'PSTB', 'PSTC', 'PSTS', 'RCNA', 'RCNR', 'ROBA', 'RPOS', 'SODA', 'SODB', 'SOXRB', 'SOXS', 'SUGE', 'TEHA', 'TEHB', 'TOLC', 'AMPH', 'YCHH', 'YDDG', 'YDEO', 'YDEP', 'YGIW', 'YGJH', 'YHCN

**https://megares.meglab.org/browse/**

### ARG-ANNOT (abricate DB)

In [12]:
argannot_summary = pd.read_csv(os.path.join(SUMMARIES_DIR, 'argannot.tab'),
                               sep='\t', na_values='.')

In [13]:
argannot_tidy = make_tidy(argannot_summary)
print(argannot_tidy)

              STRAIN                                   GENE
0     AP019703.1.tab                       (Bla)AmpC1_Ecoli
1     AP019703.1.tab                       (Bla)AmpC2_Ecoli
2     AP019703.1.tab  (Bla)Penicillin_Binding_Protein_Ecoli
3     AP019703.1.tab                        (Bla)ampH_Ecoli
4     AP019703.1.tab                           (Tet)tet(34)
..               ...                                    ...
577  chr_LREC262.tab                       (Bla)AmpC1_Ecoli
578  chr_LREC262.tab                       (Bla)AmpC2_Ecoli
579  chr_LREC262.tab  (Bla)Penicillin_Binding_Protein_Ecoli
580  chr_LREC262.tab                        (Bla)ampH_Ecoli
581  chr_LREC262.tab                           (Tet)tet(34)

[582 rows x 2 columns]


In [14]:
argannot_total_genes = argannot_tidy['GENE'].unique().tolist()
argannot_tidy_grouped = argannot_tidy.groupby(['STRAIN'])
# get genes that present in more than 95% of strains (89-93)
argannot_sheared_genes = get_sheared_entities(argannot_total_genes, 'GENE', argannot_tidy_grouped)

In [15]:
print(len(argannot_sheared_genes))
[i for i in argannot_sheared_genes]

4


['(Bla)AmpC1_Ecoli',
 '(Bla)AmpC2_Ecoli',
 '(Bla)Penicillin_Binding_Protein_Ecoli',
 '(Bla)ampH_Ecoli']

### NCBI AMR (abricate DB)

In [16]:
ncbi_summary = pd.read_csv(os.path.join(SUMMARIES_DIR, 'ncbi.tab'),
                          sep='\t', na_values='.')

In [17]:
ncbi_tidy = make_tidy(ncbi_summary)

In [18]:
ncbi_total_genes = ncbi_tidy['GENE'].unique().tolist()
ncbi_tidy_grouped = ncbi_tidy.groupby(['STRAIN'])
# get genes that present in more than 95% of strains (89-93)
ncbi_sheared_genes = get_sheared_entities(ncbi_total_genes, 'GENE', ncbi_tidy_grouped)

In [19]:
print(len(ncbi_sheared_genes))

0


### CADR (abricate DB)

In [20]:
card_summary = pd.read_csv(os.path.join(SUMMARIES_DIR, 'card.tab'),
                           sep='\t', na_values='.')

In [21]:
card_tidy = make_tidy(card_summary)

In [22]:
card_total_genes = card_tidy['GENE'].unique().tolist()
card_tidy_grouped = card_tidy.groupby(['STRAIN'])
# get genes that present in more than 95% of strains (89-93)
card_sheared_genes = get_sheared_entities(card_total_genes, 'GENE', card_tidy_grouped)

In [23]:
print(len(card_sheared_genes))
[i for i in card_sheared_genes]

44


['CRP',
 'Escherichia_coli_acrA',
 'Escherichia_coli_ampC',
 'Escherichia_coli_ampC1_beta-lactamase',
 'Escherichia_coli_ampH',
 'Escherichia_coli_mdfA',
 'H-NS',
 'Klebsiella_pneumoniae_KpnE',
 'Klebsiella_pneumoniae_KpnF',
 'Nocardia_rifampin_resistant_beta-subunit_of_RNA_polymerase_(rpoB2)',
 'acrB',
 'acrD',
 'bacA',
 'baeR',
 'baeS',
 'cpxA',
 'emrA',
 'emrB',
 'emrK',
 'emrR',
 'emrY',
 'eptA',
 'evgA',
 'evgS',
 'gadW',
 'gadX',
 'kdpE',
 'marA',
 'mdtA',
 'mdtB',
 'mdtC',
 'mdtE',
 'mdtF',
 'mdtG',
 'mdtH',
 'mdtM',
 'mdtN',
 'mdtO',
 'mdtP',
 'mphB',
 'msbA',
 'pmrF',
 'tolC',
 'yojI']

### ResFinder (abricate DB)

In [24]:
resfinder_summary = pd.read_csv(os.path.join(SUMMARIES_DIR, 'resfinder.tab'),
                                sep='\t', na_values='.')

In [25]:
resfinder_tidy = make_tidy(resfinder_summary)

In [26]:
resfinder_total_genes = resfinder_tidy['GENE'].unique().tolist()
resfinder_tidy_grouped = resfinder_tidy.groupby(['STRAIN'])
# get genes that present in more than 95% of strains (89-93)
resfinder_sheared_genes = get_sheared_entities(resfinder_total_genes, 'GENE', resfinder_tidy_grouped)

In [27]:
print(len(resfinder_sheared_genes))
[i for i in resfinder_sheared_genes]

1


['mdf(A)_1']

### Save the results

In [28]:
max_lenght = len(sheared_groups)   # 140

argannot_col = argannot_sheared_genes + [np.nan for i in range(max_lenght - len(argannot_sheared_genes))]
ncbi_col = ncbi_sheared_genes + [np.nan for i in range(max_lenght - len(ncbi_sheared_genes))]
card_col = card_sheared_genes + [np.nan for i in range(max_lenght - len(card_sheared_genes))]
resfider_col = resfinder_sheared_genes + [np.nan for i in range(max_lenght - len(resfinder_sheared_genes))]

amr_results = pd.DataFrame({'megares': sheared_groups, 'argannot': argannot_col, 'ncbi': ncbi_col,
                            'card': card_col, 'resfinder': resfider_col})
amr_results.to_csv(os.path.join(os.path.dirname(SUMMARIES_DIR), 'amr_sheared_genes.csv'),
                   header=True, index=False)